In [55]:
import tkinter as tk
from tkinter import *
from tkinter import messagebox
from tkinter import ttk
import numpy as np
import random
import time
import math

In [63]:
class Car:
    def __init__(self, length, x, y, vx, vy, ax, ay, dir, status):
        self.length = length
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.ax = ax
        self.ay = ay
        self.dir = dir
        self.gui = None
        self.max_a = 0.5
        self.max_v = 1
        self.status = status
        
    def update_pos(self, dt):
        self.x = self.x + self.vx*dt
        self.y = self.y + self.vy*dt
        self.vx = self.vx + self.ax*dt
        self.vy = self.vy + self.ay*dt
        
    def update(self, dt, light, yellowTime, int_x, int_y):
        if (self.status == "accelerating"):
            if (self.dir == 'x'):
                if (abs(self.vx + self.ax*dt) >= self.max_v):
                    self.ax = 0
                    self.vx = self.max_v
                    status = "cruising"
                    print(".")
            elif (self.dir == 'y'):
                if (abs(self.vy + self.ay*dt) >= self.max_v):
                    self.ay = 0
                    self.vy = self.max_v
                    status = "cruising"
                    print(".")
        if (self.vx * (self.vx + self.ax*dt) <= 0):
            self.vx = 0
            self.ax = 0
            self.status = "stopped"
        if (self.vy * (self.vy + self.ay*dt) <= 0):
            self.vy = 0
            self.ay = 0
            self.status = "stopped"
                    
        self.light_response(dt, light, yellowTime, int_x, int_y)
        self.update_pos(dt)
        

    def light_response(self, dt, light, yellowTime, int_x, int_y):
        if (self.dir == 'x'):
            v = self.vx
            s = self.x
            i = int_x[0]
        elif (self.dir == 'y'):
            v = self.vy
            s = self.y
            i = int_y[0]
        if (light != self.dir):
            if (s + v*yellowTime-0.5*self.max_a*yellowTime**2 > i):
                self.status = "going_on_yellow"
            elif ((s+v*dt) + v*yellowTime-0.5*self.max_a*yellowTime**2 > i):
                self.status = "braking_for_light"
                a = 2 * (i - s - v*yellowTime) / yellowTime**2
                if (self.dir == 'x'):
                    self.ax = a
                elif (self.dir == 'y'):
                    self.ay = a
        elif (self.status == "stopped"):
            self.status = "accelerating"
            if (self.dir == 'x'):
                self.ax = self.max_a
            elif (self.dir == 'y'):
                self.ay = self.max_a
        


def run_simulation(T=10000, xrate=.05, yrate=.05, dt = 1, random_speeds = False, quit_on_end=False, int_x=(240,260),int_y=(240,260), lightTime=1000, yellowTime=2):
    total_cars = 0
    light = 'x'
    root = Tk()
    root.title("Traffic Simulation")
    cv = Canvas(root,height=500, width=500)
    cv.pack()
    
    cars = []
    for t in range(T):
        if (random.random() < xrate):
            total_cars+=1
            if random_speeds:
                newCar = Car(5,0,250,(random.random()+1)/2,0,0,0,'x',"cruising")
            else:
                newCar = Car(5,0,250,1,0,0,0,'x',"cruising")
            cars.append(newCar)
        elif (random.random() < xrate + yrate):
            total_cars+=1
            if random_speeds:
                newCar = Car(5,0,250,(random.random()+1)/2,0,0,0,'x',"cruising")
            else:
                newCar = Car(5,250,0,0,1,0,0,'y',"cruising")
            cars.append(newCar)
        for car in cars:
            cv.delete(car.gui)
            car.update(dt, light, yellowTime, int_x, int_y)
            if (car.x>500):
                cars.remove(car)
            else:
                if (car.dir == 'x'):
                    car.gui = cv.create_line(car.x-car.length,car.y,car.x,car.y)
                elif (car.dir == 'y'):
                    car.gui = cv.create_line(car.x,car.y-car.length,car.x,car.y)
        root.update()
        root.after(1)

        if (t % lightTime == lightTime - 1):
                if (light == 'x'):
                    light = 'y'
                else:
                    light = 'x'
    

    if quit_on_end:
        root.destroy()

    return total_cars




In [64]:
run_simulation(T=10000,xrate=.05,yrate=.05)

AttributeError: module 'math' has no attribute 'abs'

In [ ]:
arrivals = []
for i in range(20):
    arrivals.append(run_simulation(500, quit_on_end=True))

print(arrivals)

In [80]:
arrivals = []

for j in range(20):
    input = random.random()
    cum_func = 0
    i = 0
    while (cum_func < input):
        dens_func = ((xrate*T)**i)*math.exp(-xrate*T)/math.factorial(i)
        cum_func += dens_func
        i += 1
    arrivals.append(i)

print(arrivals)

[29, 28, 35, 27, 26, 25, 22, 16, 24, 22, 30, 22, 31, 26, 37, 20, 30, 21, 22, 22]
